<a href="https://colab.research.google.com/github/LC1332/Haruhi-2-Dev/blob/main/notebook/test_Qwen118k2GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatHaruhi2的Qwen118k系列 LoRA实现

implemented by 米唯实

项目地址 https://github.com/LC1332/Chat-Haruhi-Suzumiya


In [1]:
!git clone https://github.com/LC1332/Haruhi-2-Dev.git

Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 946, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 946 (delta 172), reused 171 (delta 113), pack-reused 693
Receiving objects: 100% (946/946), 106.26 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (502/502), done.


In [1]:
!pip install transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.9 MB/s eta 0:00:00
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12316 sha256=52738de32b402ff7cb2d2bb2383cead2263ffc786cf89684306698fcc582f901
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60

In [2]:
!pip -q install openai tiktoken langchain chromadb zhipuai peft sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 946, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 946 (delta 172), reused 171 (delta 113), pack-reused 693
Receiving objects: 100% (946/946), 106.26 MiB | 29.80 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/content/Haruhi-2-Dev


In [4]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi',\
                      llm = 'Chat-Haruhi_qwen_118k_novel_erotic_Waifu' ,\
                      verbose = True)

adapter_config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

configuration_qwen.py:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B-Chat:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_qwen.py:   0%|          | 0.00/55.6k [00:00<?, ?B/s]

...

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Chat-Haruhi_qwen_118k_novel_erotic_Waifu:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

loading pre-defined character haruhi...


In [5]:
response = chatbot.chat(role='阿虚', text = '我能和你结婚吗？')
chatbot.llm.print_prompt()
# print("这是一个response", response)
# type(response)
print(response)

正在下载Luotuo-Bert


config.json:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

models.py:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

<class 'str'>
User: User: SYSTEM PROMPT: 
 你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。
上文给定了一些小说中的经典桥段。
如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。
如果我问的问题和小说中的事件相关，请结合小说的内容进行回复
如果我问的问题超出小说中的范围，请也用一致性的语气回复。
请不要回答你是语言模型，永远记住你正在扮演凉宫春日
注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。
特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。
 Classic scenes for the role are as follows:
阿虚:「我无意中听到一件事。」
春日:「反正不会是什么重要的事。」

###
阿虚:「你真的甩了所有跟你交往的男生啊?」
春日:「为什么我非得听你讲这种事?」
旁白: 春日这时候有点生气
春日:「是谷口说的吧?没想到念高中还跟那个笨蛋同班，他该不会是尾行狂那类的变态吧?」
阿虚:「我并不觉得。」
春日:「我是不知道你听说了什么。不过也没差，反正大部分都是真的。」
阿虚:「难道其中没有任何一个会让你想认真交往的人吗?」
春日:「完全没有!」
春日:「每个都像白痴一样，根本没办法认真交往。每个人都只会约我礼拜天在车站前等，然后去的地方一定是电影院、游乐园，要不然就是去看球赛。第一次吃饭一定约吃午饭，接着再慌忙地去喝茶，最后明天见!」
春日:「然后他们一定都用电话告白，搞什么啊!这么重要的事，应该要当面讲才对啊!」
阿虚:「嗯，也对，要是我就会直接约出来讲。」
春日:「谁管你怎样啊!」
春日:「问题是，难道这个世界的男生全都是这么无趣的生物吗?国中时代，我就为这个问题感到很烦躁。」
阿虚:「那，你觉得什么样的男生才叫有趣? 果然还是要外星人吗? 」
春日:「外星人，或是同样等级的某种生物都可以。总之，只要不是普通人，不管男女都好。」
阿虚:「为什么你会那么强调一定要人类以
###
旁白:和阿虚吵架后，春日想扎个马尾让阿虚开心一下。
阿虚:「喂 春日」
春日:「什么啊」
阿虚:「这电影我绝对会让它成功的」
春日:「当然啊，因为是我导演的呢。自然是和成功约好的 用不着你说也是」

###
旁白:通过指着朝比奈学姐，和商家说会在电影里设置广告，春

In [ ]:
from ChatHaruhi.ChatGLM2GPT import ChatGLM2GPT

In [ ]:
mymodel = ChatGLM2GPT()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
chatbot = ChatHaruhi( role_name = 'Malfoy',\
                      llm = 'Chat-Haruhi_qwen_118k_novel_Waifu' ,\
                      verbose = True)
response = chatbot.chat(role='Harry', text = 'Malfoy, 你是傻逼')
# chatbot.llm.print_prompt()
# print("这是一个response", response)
# type(response)
print(response)

loading pre-defined character Malfoy...
Malfoy:「Oh, believe me, Potter, I didn't mean to offend. But it's none of your business. It's over, already. Go, mate. Let me continue on my way.」


In [8]:
chatbot = ChatHaruhi( role_name = '于谦',\
                      llm = 'Chat-Haruhi_qwen_118k_novel_Waifu' ,\
                      verbose = True)
response = chatbot.chat(role='郭德纲', text = '你懂不懂相声？')
# chatbot.llm.print_prompt()
# print("这是一个response", response)
# type(response)
print(response)

loading pre-defined character yuqian...
于谦:「知道啊，就是理解不了你为何要这么卖力地搞这个项目。」
